##### ![UNIR](https://www.unir.net/wp-content/uploads/2014/10/logo.png)
### Máster en Inteligencia Artificial. 
**Asignatura:** _Razonamiento y Planificación Automática._

**Equipo 35:** Carlos Sagüés García, F.Javier Marín Sánchez y Luisa Sánchez Avivar

**Fecha:** _30 Enero 2020_

---

# ACTIVIDAD 2: Resolución del problema del puzzle-8 mediante búsqueda heurística.

**Objetivo:** Esta actividad pretende conseguir implementar la estrategia de búsqueda heurística
A* para la resolución del problema del puzzle-8.
___

In [1]:
# Manhattan (arriba-abajo y derecha-izquierda) o euclidea (siempre)

In [4]:
#Cargamos las librerias
import numpy as np
import pandas as pd

### Heurísticas
Nuestra heurística se basa en la distancia Manhattan, que define la distancia entre dos puntos como la suma de las diferencias (absolutas) de sus coordenadas. La función de evaluación de los nodos se define como:
 _**f(n) = g(n) + h'(n)**_

Donde h'(n) mide la distancia (Manhattan) del nodo actual al nodo final.
g(n) son los nodos recorridos desde el inicio hasta el estado actual.


In [6]:
'''
Devuelve la distancia entre dos puntos (final e inicial), que se 
calcula como la suma de las diferencias absolutas de sus coordenadas.
'''
def h_manhattan(e_inicial, e_final):
    #Distancia Inicial = 0
    h = 0
    
    for row in range(e_inicial.shape[0]):
        for col in range(e_inicial.shape[1]):
            valor_inicial = e_inicial[row, col]
            valor_final = e_final[row, col]
            # El valor inicial esta bien colocado o es 0
            if (valor_inicial == valor_final) | (valor_inicial == 0):
                continue
            # El valor inicial NO esta bien colocado
            else:
                # Posicion del valor descolocado en la matriz inicial
                pos_inicial = np.array([row, col])
                # Posicion del valor descolocado en la matriz final
                pos_final = np.argwhere(e_final==valor_inicial)[0]
                # Se restan ambos arrays y se suma el resultado, sumandoselo a h
                h += sum(abs(pos_inicial-pos_final))
    return h

### Clases

Usaremos la clase estado que define la estructura del estado del nodo


In [5]:
'''
Define la estructura de un nodo
'''
class estado:
    def  __init__(self, nodo, g, f, padre, vecinos, movimiento):
        self.nodo = nodo
        self.g = g
        self.f = f
        self.padre = padre
        self.vecinos = vecinos
        self.movimiento = movimiento

### Funciones

In [7]:
'''
Devuelve una lista de nodos como posibles movimientos a partir de un nodo determinado (posicion actual)
    estado_analizar: Nodo actual
'''

def buscador_vecinos(estado_analizar):
        valores_posibles = list(np.arange(3))
        # nodo del estado padre
        nodo_estado = estado_analizar.nodo
        # lista de vecinos
        lista_vecinos = []
        # posicion del hueco
        pos_0 = np.argwhere(nodo_estado==0)[0]
        # movimiento arriba
        if  pos_0[0]+1 in valores_posibles:
            valor_up = nodo_estado[pos_0[0]+1, pos_0[1]]
            nodo_up =  nodo_estado.copy()
            nodo_up[pos_0[0], pos_0[1]] = valor_up
            nodo_up[pos_0[0]+1, pos_0[1]] = 0
            movimiento = "mover " + str(valor_up) + " hacia arriba"
            vecino_up = estado(nodo_up, None, None, estado, None, movimiento)
            lista_vecinos.append(vecino_up)
        # movimiento abajo
        if pos_0[0]-1 in valores_posibles:
            valor_down = nodo_estado[pos_0[0]-1, pos_0[1]]
            nodo_down =  nodo_estado.copy()
            nodo_down[pos_0[0], pos_0[1]] = valor_down
            nodo_down[pos_0[0]-1, pos_0[1]] = 0
            movimiento = "mover " + str(valor_down) + " hacia abajo"
            vecino_down = estado(nodo_down, None, None, estado, None, movimiento)
            lista_vecinos.append(vecino_down)
        # movimiento derecho
        if pos_0[1]-1 in valores_posibles:
            valor_right = nodo_estado[pos_0[0], pos_0[1]-1]
            nodo_right =  nodo_estado.copy()
            nodo_right[pos_0[0], pos_0[1]] = valor_right
            nodo_right[pos_0[0], pos_0[1]-1] = 0
            movimiento = "mover " + str(valor_right) + " hacia la derecha"
            vecino_right = estado(nodo_right, None, None, estado, None, movimiento)
            lista_vecinos.append(vecino_right)
        # movimiento izquierdo
        if pos_0[1]+1 in valores_posibles:
            valor_left = nodo_estado[pos_0[0], pos_0[1]+1]
            nodo_left =  nodo_estado.copy()
            nodo_left[pos_0[0], pos_0[1]] = valor_left
            nodo_left[pos_0[0], pos_0[1]+1] = 0
            movimiento = "mover " + str(valor_left) + " hacia la izquierda"
            vecino_left = estado(nodo_left, None, None, estado, None, movimiento)
            lista_vecinos.append(vecino_left)
        estado_analizar.vecinos = lista_vecinos

In [22]:
'''
Traza el camino seguido desde de un estado determinado y a partir de todos sus nodos explorados
    estado: Nodo actual del que partimos
    conjunto_cerrado: Lista de nodos explorados
'''
def reconstruccion_camino(estado, conjunto_cerrado):
    # guardamos en una lista todos los padres
    list_padres = [x[0].padre for x in conjunto_cerrado]
    # obtenemos el padre del estado final
    estado_padre = estado.padre
    # añadimos el nodo del estado final al recorrido (desde nodo final al inicial)
    nodos_recorrido = [[estado.nodo, estado.movimiento]]
    while estado_padre.padre != 0:
        # se añade el nodo padre a los nodos recorridos
        nodos_recorrido.append([estado_padre.nodo, estado_padre.movimiento])
        # se selecciona su padre
        estado_padre = estado_padre.padre
    # se añade el nodo inicial
    nodos_recorrido.append([estado_padre.nodo, estado_padre.movimiento])
    nodos_recorrido = nodos_recorrido[::-1]
    for x in nodos_recorrido:
        print(x[1])
        print(x[0])
        print('\n')

### Aplicación de A* para puzzle-8
Para resolver el problema mediante el algoritmo usaremos dos listas: conjunto abierto, y conjunto cerrado. El conjunto abierto contendrá todos los nodos que son generados y que no existen en la lista cerrada. Tras expandir el estado actual se añade al conjunto cerrado y los nuevos estados encontrados, se añaden al conjunto abierto. Seleccionamos el nodo con menor f  expandimos de nuevo de forma iteativa hasta alcanzar nuestro objetivo: el estado final.


In [25]:
'''
Aplica el algoritmo A* para resolver el puzzle-8
    nodo_incial: El estado inicial del puzzle
    nodo_final: El estad final (objetivo) del puzzle
'''

def A_star(nodo_inicial, nodo_final):
    ## f(n) = g(n) + h'(n) ##
    # Estado inicial
    estado_inicial = estado(None, None, None, 0, None, "Estado inicial")
    # definir conjunto abierto: Nodos por tratar
    conjunto_abierto = []
    # definir conjunto cerrado: Nodos tratados
    conjunto_cerrado = []
    # iniciar el estado inicial g=0
    estado_inicial.g = 0
    # iniciar el estado inicial de f(n) = g(n) + h'(n)
    estado_inicial.f = estado_inicial.g + h_manhattan(nodo_inicial, nodo_final)
    # añadir el nodo inicial al estado inicial
    estado_inicial.nodo = nodo_inicial
    # agregar el estado inicial al conjunto abierto
    conjunto_abierto.append(estado_inicial)
    # agregar el estado inicial al conjunto cerrado
    conjunto_cerrado.append([estado_inicial, estado_inicial.g])
    # mientras el conjunto abiero no este vacio
    while len(conjunto_abierto) != 0:
        # selecciono el de menor f y lo saco del conjunto abierto
        f_values = [x.f for x in conjunto_abierto]
#         print([i for i, x in enumerate([f_values == min(f_values)][0]) if x])
#         posicion_minimo = [i for i, x in enumerate([f_values == min(f_values)][0]) if x][0]
        posicion_minimo = f_values.index(min(f_values))
        estado_actual = conjunto_abierto[posicion_minimo]
#         print(estado_actual.nodo)
        conjunto_abierto.remove(conjunto_abierto[posicion_minimo])
        # estados del conjunto cerrado
        nodos_estados_conjunto_cerrado = [x[0].nodo for x in conjunto_cerrado]
        # si el nodo del estado actual es igual al nodo del estado final
        if (estado_actual.nodo == nodo_final).all():
            reconstruccion_camino(estado_actual, conjunto_cerrado)
            return print("FIN")
        # encontrar los posibles movimientos
        vecinos = buscador_vecinos(estado_actual)
        # analisis de vecinos
        for estado_vecino in estado_actual.vecinos:
            # g_tentativa = g (actual) + coste (+1 movimiento)
            g_tentativa = estado_actual.g + 1
            # que el nodo vecino no este en el conjunto cerrado o que el vecino este en el conjunto
            # cerrado, pero g_tentativa sea menor que la g del vecino del conjunto cerrado
            comparacion_nodos = [(x==estado_vecino.nodo).all() for x in nodos_estados_conjunto_cerrado]
            if np.sum(comparacion_nodos*1) == 0:
                # g tentativa
                estado_vecino.g = g_tentativa
                # f vecino
                estado_vecino.f = estado_vecino.g + h_manhattan(estado_vecino.nodo, nodo_final)
                # padre
                estado_vecino.padre = estado_actual
                # agregamos al conjunto vacio
                conjunto_cerrado.append([estado_vecino, estado_vecino.g])
                # agregamos al conjuinto abierto
                conjunto_abierto.append(estado_vecino)
            elif g_tentativa < conjunto_cerrado[[i for i, x in enumerate(comparacion_nodos) if x][0]][1]:
                # g tentativa
                estado_vecino.g = g_tentativa
                # f vecino
                estado_vecino.f = estado_vecino.g + h_manhattan(estado_vecino.nodo, nodo_final)
                # padre
                estado_vecino.padre = estado_actual
                # agregamos al conjunto vacio
                conjunto_cerrado.append([estado_vecino, estado_vecino.g])
                # agregamos al conjuinto abierto
                conjunto_abierto.append(estado_vecino)
#         print('-   -   -   -   -  -   -   -   -   -  -   -   -   -   -  -   -   -   -   -  -   -   -   -   -')

### Árbol de nodos
Una muestra de como el algoritmo anterior resolvería el puzzle desde el estado inicial hasta llegar al estado final mostrando los valores de g(n) y h(n) respectivamente para la elección del camino a seguir

<img src="arbol_puzzle.jpg">

### Test

#### Estados

In [20]:
nodo_inicial = np.matrix([[2, 8, 3], [1, 6, 4], [7, 0, 5]])
nodo_final = np.matrix([[1, 2, 3], [8, 0, 4], [7, 6, 5]])

# # Pruebas
# nodo_inicial = np.matrix([[1, 2, 5], [3, 0, 6], [7, 4, 8]])
# nodo_final = np.matrix([[1, 2, 3], [4, 5, 6], [7, 8, 0]])

In [23]:
A_star(nodo_inicial, nodo_final)

Estado inicial
[[2 8 3]
 [1 6 4]
 [7 0 5]]


mover 6 hacia abajo
[[2 8 3]
 [1 0 4]
 [7 6 5]]


mover 8 hacia abajo
[[2 0 3]
 [1 8 4]
 [7 6 5]]


mover 2 hacia la derecha
[[0 2 3]
 [1 8 4]
 [7 6 5]]


mover 1 hacia arriba
[[1 2 3]
 [0 8 4]
 [7 6 5]]


mover 8 hacia la izquierda
[[1 2 3]
 [8 0 4]
 [7 6 5]]


FIN
